In [2]:
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error
from sklearn.preprocessing import StandardScaler,  OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
df = pd.read_csv("data/train.csv")

df['Weight_per_Age'] = df['Weight'] / (df['Age'] + 1)

# HeartRate per Weight
df['HeartRate_per_kg'] = df['Heart_Rate'] / df['Weight']

# Duration Per Age
df['Duration_per_age'] = df['Duration'] / (df['Age'] + 1)

# Duration * Heart Rate
df['Duration_heart_rate'] = df['Duration']*df['Heart_Rate']

# Intensity
df['Duration_per_weight'] = df['Duration']/df['Weight']

# All Durations add and multi
df['duration_sum'] = df['Duration_per_weight'] + \
    df['Duration_heart_rate']+df['Duration_per_age']
df['duration_multi'] = df['Duration_per_weight'] * \
    df['Duration_heart_rate']*df['Duration_per_age']

# Creating new column 'BMI'
df['BMI'] = df['Weight']/(df['Height'] ** 2)
df['BMI'] = df['BMI'].round(2)


X = df.drop(columns=["id", "Calories"])

y = np.log(df['Calories'])

numerical_features = [col for col in X.columns if col not in ["Sex"]]

categorical_features = ["Sex"]
X['Sex'] = X['Sex'].map({'female': 1, 'male': 0})

X

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Weight_per_Age,HeartRate_per_kg,Duration_per_age,Duration_heart_rate,Duration_per_weight,duration_sum,duration_multi,BMI
0,0,36,189.0,82.0,26.0,101.0,41.0,2.216216,1.231707,0.702703,2626.0,0.317073,2627.019776,585.094265,0.0
1,1,64,163.0,60.0,8.0,85.0,39.7,0.923077,1.416667,0.123077,680.0,0.133333,680.256410,11.158974,0.0
2,1,51,161.0,64.0,7.0,84.0,39.8,1.230769,1.312500,0.134615,588.0,0.109375,588.243990,8.657452,0.0
3,0,20,192.0,90.0,25.0,105.0,40.7,4.285714,1.166667,1.190476,2625.0,0.277778,2626.468254,868.055556,0.0
4,1,38,166.0,61.0,25.0,102.0,40.6,1.564103,1.672131,0.641026,2550.0,0.409836,2551.050862,669.924338,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,0,28,193.0,97.0,30.0,114.0,40.9,3.344828,1.175258,1.034483,3420.0,0.309278,3421.343761,1094.205475,0.0
749996,1,64,165.0,63.0,18.0,92.0,40.5,0.969231,1.460317,0.276923,1656.0,0.285714,1656.562637,131.024176,0.0
749997,0,60,162.0,67.0,29.0,113.0,40.9,1.098361,1.686567,0.475410,3277.0,0.432836,3277.908246,674.322731,0.0
749998,0,45,182.0,91.0,17.0,102.0,40.3,1.978261,1.120879,0.369565,1734.0,0.186813,1734.556378,119.714763,0.0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)



numerical_features = [col for col in X.columns if col not in ["Sex"]]


# Create a ColumnTransformer to apply different preprocessing strategies

preprocessor = ColumnTransformer(

    transformers=[

        ("num", StandardScaler(), numerical_features),

    ],
    remainder = "passthrough"

).fit(X)


poly = PolynomialFeatures(2, interaction_only=True)

X_train = preprocessor.transform(X)
X_test = preprocessor.transform(X_test)

poly.fit(X_train)

X_train = poly.transform(X_train)
X_test = poly.transform(X_test)

In [7]:
import optuna
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score

# Define an objective function for Optuna to optimize
def objective(trial):
    # Define the hyperparameters to optimize
    params = {
        'tree_method': 'gpu_hist',
        'gpu_id': 0,
        'predictor': 'gpu_predictor',
        'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 5, 16),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5)
    }
    
    # Create the model with the suggested hyperparameters
    model = XGBRegressor(**params)
    
    # Perform cross-validation
    scores = cross_val_score(
        model, 
        X_train, 
        y, 
        cv=5, 
        scoring='neg_mean_squared_error',
        n_jobs=1  # Use 1 when using GPU to avoid conflicts
    )
    
    # Return the mean negative score (Optuna minimizes the objective)
    return -scores.mean()

# Create a study object and optimize
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
    sampler=optuna.samplers.TPESampler(seed=42)
)

# Run optimization with parallelization
study.optimize(objective, n_trials=30, n_jobs=1, show_progress_bar=True)

# Get the best parameters
best_params = study.best_params
print("Best parameters:", best_params)
print(f"Best MSLE: {-study.best_value}")

# Create a model with the best parameters
best_model = XGBRegressor(
    tree_method='gpu_hist',
    gpu_id=0,
    predictor='gpu_predictor',
    **best_params
)

# Train the model with the best parameters
best_model.fit(X_train, y)

# Plot optimization history
try:
    from optuna.visualization import plot_optimization_history, plot_param_importances
    
    # Plot the optimization history
    history_plot = plot_optimization_history(study)
    
    # Plot parameter importances
    importance_plot = plot_param_importances(study)
    
    # Print a message (these plots would be visible in a notebook environment)
    print("Optimization history and parameter importance plots created.")
    history_plot.write_html("plot/optimization_history.html")
    importance_plot.write_html("plot/parameter_importance.html")

except ImportError:
    print("Optuna visualization tools not available. Install with: pip install optuna[visualization]")



[I 2025-05-15 19:53:59,812] A new study created in memory with name: no-name-8747f6a7-0aa1-4534-bac5-a60d67f5fa44
  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: UserWarning:

[19:54:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: UserWarning:

[19:54:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\core.py:2676: UserWarning:

[19:54:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` paramet

[I 2025-05-15 19:55:01,312] Trial 0 finished with value: 0.004700209031929336 and parameters: {'n_estimators': 2185, 'learning_rate': 0.17254716573280354, 'max_depth': 13, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'gamma': 0.2904180608409973}. Best is trial 0 with value: 0.004700209031929336.


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: UserWarning:

[19:55:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: UserWarning:

[19:55:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\core.py:2676: UserWarning:

[19:55:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: Us

[I 2025-05-15 19:56:57,914] Trial 1 finished with value: 0.004669198030980955 and parameters: {'n_estimators': 4398, 'learning_rate': 0.06054365855469246, 'max_depth': 13, 'min_child_weight': 1, 'subsample': 0.9849549260809971, 'colsample_bytree': 0.9162213204002109, 'gamma': 1.0616955533913808}. Best is trial 1 with value: 0.004669198030980955.


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: UserWarning:

[19:56:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: UserWarning:

[19:56:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\core.py:2676: UserWarning:

[19:57:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: Us

...

[I 2025-05-15 20:37:50,405] Trial 28 finished with value: 0.004694020131681197 and parameters: {'n_estimators': 2883, 'learning_rate': 0.05130883253208271, 'max_depth': 14, 'min_child_weight': 2, 'subsample': 0.7523583588778976, 'colsample_bytree': 0.83230655973243, 'gamma': 1.3291665417366727}. Best is trial 15 with value: 0.004575996005608951.


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: UserWarning:

[20:37:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: UserWarning:

[20:37:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\core.py:2676: UserWarning:

[20:38:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: Us

[I 2025-05-15 20:39:09,633] Trial 29 finished with value: 0.004552157957400124 and parameters: {'n_estimators': 2255, 'learning_rate': 0.030410937809855695, 'max_depth': 13, 'min_child_weight': 6, 'subsample': 0.8104915666350779, 'colsample_bytree': 0.8904557858312604, 'gamma': 0.31838966282284686}. Best is trial 29 with value: 0.004552157957400124.
Best parameters: {'n_estimators': 2255, 'learning_rate': 0.030410937809855695, 'max_depth': 13, 'min_child_weight': 6, 'subsample': 0.8104915666350779, 'colsample_bytree': 0.8904557858312604, 'gamma': 0.31838966282284686}
Best MSLE: -0.004552157957400124


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: UserWarning:

[20:39:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"


c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\training.py:183: UserWarning:

[20:39:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.




Optimization history and parameter importance plots created.


In [8]:
y_pred = best_model.predict(X_test)
rmsle_score = mean_squared_log_error(np.exp(y_test), np.exp(y_pred))
print(f"Optimized Stacking Ensemble RMSLE: {rmsle_score:.4f}")
r2 = r2_score(np.exp(y_test), np.exp(y_pred))
print(f"R-squared Score: {r2}")

c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\xgboost\core.py:2676: UserWarning:

[22:43:27] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"




Optimized Stacking Ensemble RMSLE: 0.0031
R-squared Score: 0.9968519853821007


In [9]:
X_submission = pd.read_csv("data/test.csv")

# Weight_per_Age
X_submission['Weight_per_Age'] = X_submission['Weight'] / (X_submission['Age'] + 1)

# HeartRate per Weight
X_submission['HeartRate_per_kg'] = X_submission['Heart_Rate'] / X_submission['Weight']

# Duration Per Age
X_submission['Duration_per_age'] = X_submission['Duration'] / (X_submission['Age'] + 1)

# Duration * Heart Rate
X_submission['Duration_heart_rate']=X_submission['Duration']*X_submission['Heart_Rate']

# Intensity
X_submission['Duration_per_weight']=X_submission['Duration']/X_submission['Weight']

# All Durations add and multi
X_submission['duration_sum']=X_submission['Duration_per_weight']+X_submission['Duration_heart_rate']+X_submission['Duration_per_age']
X_submission['duration_multi']=X_submission['Duration_per_weight']*X_submission['Duration_heart_rate']*X_submission['Duration_per_age']

# Creating new column 'BMI'
X_submission['BMI']=X_submission['Weight']/((X_submission['Height']/100) ** 2)
X_submission['BMI']=X_submission['BMI'].round(2)

X_submission['Sex'] = X_submission['Sex'].map({'female': 1, 'male': 0})

out=X_submission[["id"]]
X_submission = preprocessor.transform(X_submission.drop(columns=["id"]))
X_submission = poly.transform(X_submission)
y_out =  best_model.predict(X_submission)
out["Calories"] = np.exp(y_out)
out.to_csv("data/submission_xgb.csv", index=False)
out

,id,Calories
0,750000,27.580971
1,750001,108.314293
2,750002,86.893906
3,750003,127.035629
4,750004,76.142113
...,...,...
249995,999995,26.037237
249996,999996,9.734456
249997,999997,73.259346
249998,999998,168.674973


In [ ]:
import optuna
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score

# Define an objective function for Optuna to optimize
def objective(trial):
    # Define the hyperparameters to optimize
    params = {
        'boosting_type': 'gbdt',
        'device': 'gpu',  # Use GPU for acceleration
        'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 5, 16),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 5),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 5)
    }
    
    # Create the model with the suggested hyperparameters
    model = LGBMRegressor(**params)
    
    # Perform cross-validation
    scores = cross_val_score(
        model, 
        X_train, 
        y, 
        cv=5, 
        scoring='neg_mean_squared_error',
        n_jobs=1  # Use 1 when using GPU to avoid conflicts
    )
    
    # Return the mean negative score (Optuna minimizes the objective)
    return -scores.mean()

# Create a study object and optimize
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
    sampler=optuna.samplers.TPESampler(seed=42)
)

# Run optimization with parallelization
study.optimize(objective, n_trials=30, n_jobs=1, show_progress_bar=True)

# Get the best parameters
best_params = study.best_params
print("Best parameters:", best_params)
print(f"Best MSLE: {-study.best_value}")

# Create a model with the best parameters
best_model = LGBMRegressor(
    boosting_type='gbdt',
    device='gpu',
    **best_params
)

# Train the model with the best parameters
best_model.fit(X_train, y)

# Plot optimization history
try:
    from optuna.visualization import plot_optimization_history, plot_param_importances
    
    # Plot the optimization history
    history_plot = plot_optimization_history(study)
    
    # Plot parameter importances
    importance_plot = plot_param_importances(study)
    
    # Print a message (these plots would be visible in a notebook environment)
    print("Optimization history and parameter importance plots created.")
    history_plot.write_html("plot/optimization_history_lightgbm.html")
    importance_plot.write_html("plot/parameter_importance_lightgbm.html")

except ImportError:
    print("Optuna visualization tools not available. Install with: pip install optuna[visualization]")

[I 2025-05-15 22:54:24,789] A new study created in memory with name: no-name-6f6a4bca-2b36-4623-bd93-35a90b625e60
  0%|          | 0/30 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=0.2904180608409973, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2904180608409973
[LightGBM] [Warning] lambda_l2 is set=4.330880728874676, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.330880728874676
[LightGBM] [Warning] lambda_l1 is set=0.2904180608409973, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2904180608409973
[LightGBM] [Warning] lambda_l2 is set=4.330880728874676, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.330880728874676
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 24125
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 105
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4070 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 105 dense feature groups (61.80 

c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=0.2904180608409973, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2904180608409973
[LightGBM] [Warning] lambda_l2 is set=4.330880728874676, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.330880728874676
[LightGBM] [Warning] lambda_l1 is set=0.2904180608409973, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2904180608409973
[LightGBM] [Warning] lambda_l2 is set=4.330880728874676, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.330880728874676
[LightGBM] [Warning] lambda_l1 is set=0.2904180608409973, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2904180608409973
[LightGBM] [Warning] lambda_l2 is set=4.330880728874676, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.330880728874676
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 24124
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 105
[LightGBM] [Info] Using GPU Dev

c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


...

[LightGBM] [Warning] lambda_l1 is set=2.3521562812086163, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.3521562812086163
[LightGBM] [Warning] lambda_l2 is set=4.907087178685384, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.907087178685384
[LightGBM] [Warning] lambda_l1 is set=2.3521562812086163, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.3521562812086163
[LightGBM] [Warning] lambda_l2 is set=4.907087178685384, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.907087178685384
[LightGBM] [Warning] lambda_l1 is set=2.3521562812086163, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.3521562812086163
[LightGBM] [Warning] lambda_l2 is set=4.907087178685384, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.907087178685384
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 24124
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 105
[LightGBM] [Info] Using GPU Dev

c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=2.3521562812086163, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.3521562812086163
[LightGBM] [Warning] lambda_l2 is set=4.907087178685384, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.907087178685384
[LightGBM] [Warning] lambda_l1 is set=2.3521562812086163, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.3521562812086163
[LightGBM] [Warning] lambda_l2 is set=4.907087178685384, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.907087178685384
[LightGBM] [Warning] lambda_l1 is set=2.3521562812086163, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.3521562812086163
[LightGBM] [Warning] lambda_l2 is set=4.907087178685384, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.907087178685384
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 24122
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 105
[LightGBM] [Info] Using GPU Dev

c:\Users\Nicholas\anaconda3\envs\LLM\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=2.3521562812086163, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.3521562812086163
[LightGBM] [Warning] lambda_l2 is set=4.907087178685384, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.907087178685384
[LightGBM] [Warning] lambda_l1 is set=2.3521562812086163, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.3521562812086163
[LightGBM] [Warning] lambda_l2 is set=4.907087178685384, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.907087178685384
[LightGBM] [Warning] lambda_l1 is set=2.3521562812086163, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.3521562812086163
[LightGBM] [Warning] lambda_l2 is set=4.907087178685384, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.907087178685384
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 24124
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 105
[LightGBM] [Info] Using GPU Dev

In [ ]:
y_pred = best_model.predict(X_test)
rmsle_score = mean_squared_log_error(np.exp(y_test), np.exp(y_pred))
print(f"Optimized Stacking Ensemble RMSLE: {rmsle_score:.4f}")
r2 = r2_score(np.exp(y_test), np.exp(y_pred))
print(f"R-squared Score: {r2}")

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Example predictions from different models (rows: samples, columns: models)
predictions = np.array([
    [0.8, 0.7, 0.75],  # Model 1, Model 2, Model 3 predictions for sample 1
    [0.6, 0.65, 0.63],  # Sample 2
    [0.9, 0.85, 0.88]   # Sample 3
])

# True labels
true_values = np.array([0.78, 0.62, 0.87])

# Objective function: minimize weighted error
def objective(weights):
    weighted_preds = np.dot(predictions, weights)
    error = np.mean((weighted_preds - true_values) ** 2)  # Mean squared error
    return error

# Constraints: weights sum to 1
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

# Bounds: weights should be between 0 and 1
bounds = [(0, 1)] * predictions.shape[1]

# Initial weights
initial_weights = np.ones(predictions.shape[1]) / predictions.shape[1]

# Optimize weights
result = minimize(objective, initial_weights, bounds=bounds, constraints=constraints)

optimal_weights = result.x
print("Optimal Weights:", optimal_weights)